# Spark streaming: Kafka example

This is a simple example of how to run Spark Streaming jobs in Jupyter reading from Kafka.

To run this example, you should first start the Kafka and Publisher provided for project 2.

In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import socket
from pyspark.streaming.kafka import KafkaUtils

sc = SparkContext("local[2]", "KafkaExample")
print("SparkContext created!")
ssc = StreamingContext(sc, 5)
print("StreamingContext created!")


SparkContext created!
StreamingContext created!


In [2]:

lines = KafkaUtils.createDirectStream(ssc, ["debs"], \
            {"metadata.broker.list": "kafka:9092"})
# tut query

lines = lines.window(15,5)

lines.pprint()
ssc.start()

In [ ]:
ssc.stop()
sc.stop()